# CMeEE-V2 数据预处理：JSON → BIO格式

本notebook完成以下任务：
1. 加载和探索原始JSON数据
2. 理解BIO标注格式
3. 处理嵌套实体问题
4. 转换为BIO格式
5. 使用BERT Tokenizer对齐标签
6. 保存处理后的数据


## 1. 导入必要的库


In [1]:
import json
import pandas as pd
from collections import Counter, defaultdict
import random
from typing import List, Dict, Tuple

# 设置随机种子
random.seed(2025)

print("✅ 基础库导入成功")


✅ 基础库导入成功


## 2. 加载原始数据


In [2]:
# 加载三个数据集
with open('CMeEE-V2_train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

with open('CMeEE-V2_dev.json', 'r', encoding='utf-8') as f:
    dev_data = json.load(f)

with open('CMeEE-V2_test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

print(f"训练集样本数: {len(train_data)}")
print(f"验证集样本数: {len(dev_data)}")
print(f"测试集样本数: {len(test_data)}")
print(f"总计: {len(train_data) + len(dev_data) + len(test_data)}")


训练集样本数: 15000
验证集样本数: 5000
测试集样本数: 3000
总计: 23000


In [3]:
# 查看第一个样本
print("【第一个训练样本】")
sample = train_data[0]
print(f"文本: {sample['text']}")
print(f"\n实体列表:")
for ent in sample['entities']:
    print(f"  - [{ent['type']}] {ent['entity']} (位置: {ent['start_idx']}-{ent['end_idx']})")


【第一个训练样本】
文本: （5）房室结消融和起搏器植入作为反复发作或难治性心房内折返性心动过速的替代疗法。

实体列表:
  - [pro] 房室结消融 (位置: 3-8)
  - [pro] 起搏器植入 (位置: 9-14)
  - [dis] 反复发作或难治性心房内折返性心动过速 (位置: 16-34)
  - [pro] 替代疗法 (位置: 35-39)


In [4]:
# 统计实体类型分布
entity_types = Counter()
total_entities = 0
text_lengths = []
entities_per_sample = []

for item in train_data:
    text_lengths.append(len(item['text']))
    if 'entities' in item:
        num_ents = len(item['entities'])
        total_entities += num_ents
        entities_per_sample.append(num_ents)
        for ent in item['entities']:
            entity_types[ent['type']] += 1

print("\n【实体类型统计】")
print(f"总实体数: {total_entities}")
print(f"\n各类型分布:")
for ent_type, count in entity_types.most_common():
    percentage = count / total_entities * 100
    print(f"  {ent_type:6s}: {count:6d} ({percentage:5.2f}%)")

print(f"\n【文本长度统计】")
print(f"平均长度: {sum(text_lengths) / len(text_lengths):.1f} 字符")
print(f"最短: {min(text_lengths)}, 最长: {max(text_lengths)}")
print(f"中位数: {sorted(text_lengths)[len(text_lengths)//2]}")

print(f"\n【每个样本的平均实体数】")
print(f"平均: {sum(entities_per_sample) / len(entities_per_sample):.2f}")
print(f"最少: {min(entities_per_sample)}, 最多: {max(entities_per_sample)}")



【实体类型统计】
总实体数: 82649

各类型分布:
  bod   :  24106 (29.17%)
  dis   :  19371 (23.44%)
  sym   :  17118 (20.71%)
  pro   :   9688 (11.72%)
  ite   :   4410 ( 5.34%)
  dru   :   4379 ( 5.30%)
  mic   :   2343 ( 2.83%)
  equ   :    889 ( 1.08%)
  dep   :    345 ( 0.42%)

【文本长度统计】
平均长度: 54.2 字符
最短: 4, 最长: 4870
中位数: 43

【每个样本的平均实体数】
平均: 5.51
最少: 0, 最多: 99


## 4. 检测嵌套实体问题


In [5]:
# 检查是否有嵌套实体
def check_nested_entities(data):
    """检查数据中的嵌套实体"""
    nested_count = 0
    nested_examples = []
    
    for item in data:
        entities = item.get('entities', [])
        if len(entities) < 2:
            continue
            
        # 检查任意两个实体是否重叠
        for i in range(len(entities)):
            for j in range(i+1, len(entities)):
                e1, e2 = entities[i], entities[j]
                # 检查是否重叠
                if not (e1['end_idx'] <= e2['start_idx'] or e2['end_idx'] <= e1['start_idx']):
                    nested_count += 1
                    if len(nested_examples) < 5:  # 保存前5个例子
                        nested_examples.append({
                            'text': item['text'],
                            'entity1': e1,
                            'entity2': e2
                        })
    
    return nested_count, nested_examples

nested_count, nested_examples = check_nested_entities(train_data)

print(f"【嵌套实体检测】")
print(f"发现 {nested_count} 个嵌套/重叠实体对\n")

if nested_examples:
    print("嵌套实体示例：\n")
    for i, example in enumerate(nested_examples[:3], 1):
        print(f"示例 {i}:")
        print(f"  文本: {example['text']}")
        e1, e2 = example['entity1'], example['entity2']
        print(f"  实体1: [{e1['type']}] '{e1['entity']}' (位置 {e1['start_idx']}-{e1['end_idx']})")
        print(f"  实体2: [{e2['type']}] '{e2['entity']}' (位置 {e2['start_idx']}-{e2['end_idx']})")
        print()


【嵌套实体检测】
发现 15166 个嵌套/重叠实体对

嵌套实体示例：

示例 1:
  文本: 病程早期可闻胸膜摩擦音在全部呼吸期间均可听到。
  实体1: [sym] '闻胸膜摩擦音在全部呼吸期间均可听到' (位置 5-22)
  实体2: [bod] '胸膜' (位置 6-8)

示例 2:
  文本: 胸部X线透视和胸片可见患侧膈呼吸运动减弱肋膈角变钝流行性胸痛和带状疱疹前驱期的胸痛及肋骨骨折相鉴别。
  实体1: [pro] '胸部X线透视' (位置 0-6)
  实体2: [sym] '胸部X线透视和胸片可见患侧膈呼吸运动减弱' (位置 0-20)

示例 3:
  文本: 胸部X线透视和胸片可见患侧膈呼吸运动减弱肋膈角变钝流行性胸痛和带状疱疹前驱期的胸痛及肋骨骨折相鉴别。
  实体1: [sym] '胸部X线透视和胸片可见患侧膈呼吸运动减弱' (位置 0-20)
  实体2: [pro] '胸片' (位置 7-9)



## 5. BIO格式转换核心函数


In [6]:
def convert_to_bio(text: str, entities: List[Dict], strategy: str = 'longest') -> List[str]:
    """
    将span格式的实体转换为BIO标注格式
    
    Args:
        text: 原始文本
        entities: 实体列表 [{start_idx, end_idx, type, entity}, ...]
        strategy: 处理嵌套实体的策略
            - 'longest': 保留最长的实体
            - 'first': 按出现顺序，先标注的优先
    
    Returns:
        BIO标签列表，与text中的字符一一对应
    """
    # 初始化所有位置为 O
    labels = ['O'] * len(text)
    
    if not entities:
        return labels
    
    # 处理嵌套：按长度降序排序，优先处理长实体
    if strategy == 'longest':
        entities_sorted = sorted(entities, 
                                key=lambda x: x['end_idx'] - x['start_idx'], 
                                reverse=True)
    else:
        entities_sorted = entities
    
    # 标注实体
    for entity in entities_sorted:
        start = entity['start_idx']
        end = entity['end_idx']
        ent_type = entity['type']
        
        # 检查起始位置是否已被标注
        if labels[start] == 'O':
            # 标注开始位置
            labels[start] = f'B-{ent_type}'
            
            # 标注内部位置
            for i in range(start + 1, end):
                if labels[i] == 'O':  # 只标注未被占用的位置
                    labels[i] = f'I-{ent_type}'
    
    return labels

print("✅ BIO转换函数定义完成")


✅ BIO转换函数定义完成
